In [ ]:
import pickle as p
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import FormatStrFormatter
import pathlib
from scipy import stats
import matplotlib
from scipy.stats import ks_2samp
matplotlib.rcParams['pdf.fonttype'] = 42
matplotlib.rcParams['ps.fonttype'] = 42
plt.rc('legend', fontsize=18)
plt.rc('xtick', labelsize=14)
plt.rc('ytick', labelsize=14)
plt.rc('axes', labelsize=22)

## Summarize Results

In [ ]:
def summarize_res_evaluation_phase(list_variants):
    for variant in list_variants:
        path_file = f'{PATH_RESULTS}/{problem}/{problem}_{variant}'

        nEvals_list = []
        search_cost_list = []

        EAs_list = []
        IGD_list = []
        IGDp_list = []
        HV_list = []
        best_arch_list = []
        best_performance_list = []

        for rid in range(31):
            EAs_history = []
            best_arch_history = []
            best_performance_history = []

            p_file_ = path_file + f'/{rid}'

            try:
                nEvals_history, EA_evaluate_history = p.load(open(p_file_ + '/#Evals_and_Elitist_Archive_evaluate.p', 'rb'))
                _, IGD_history = p.load(open(p_file_ + '/#Evals_and_IGD.p', 'rb'))
                _, IGDp_history = p.load(open(p_file_ + '/#Evals_and_IGDp.p', 'rb'))
                _, HV_history = p.load(open(p_file_ + '/#Evals_and_HV.p', 'rb'))
            except FileNotFoundError:
                raise FileNotFoundError('Please download the prepared results!')
            search_cost_history = p.load(open(p_file_ + '/running_time.p', 'rb'))

            # Add the final evaluation_mark (i.e., 3000)
            if nEvals_history[-1] != len(search_cost_history):
                nEvals_history.append(len(search_cost_history))
                EA_evaluate_history.append(EA_evaluate_history[-1])
                IGD_history.append(IGD_history[-1])
                IGDp_history.append(IGDp_history[-1])
                HV_history.append(HV_history[-1])

            nEvals_history = np.array(nEvals_history)
            IGD_history = np.array(IGD_history)
            IGDp_history = np.array(IGDp_history)
            HV_history = np.array(HV_history)
            search_cost_history = np.array(search_cost_history)

            for EA in EA_evaluate_history:
                F = np.round(EA['F'], 6)
                ea = {
                    'Approximation Set': EA['X'],
                    'hashKey': EA['hashKey'],
                    'Approximation Front': F
                }
                idx_best_arch = np.argmin(F[:, 0])
                EAs_history.append(ea)
                best_arch = EA['X'][idx_best_arch]
                best_performance = np.round((1 - EA['F'][idx_best_arch][0]) * 100, 2)
                best_arch_history.append(best_arch)
                best_performance_history.append(best_performance)

            new_nEvals_history = np.arange(20, 3001, 20)
            new_IGD_history = []
            new_IGDp_history = []
            new_HV_history = []
            new_EAs_history = []
            new_best_arch_history = []
            new_best_performance_history = []
            for nEvals in new_nEvals_history:
                idx = np.where(nEvals_history <= nEvals)[0][-1]
                new_IGD_history.append(IGD_history[idx])
                new_IGDp_history.append(IGDp_history[idx])
                new_HV_history.append(HV_history[idx])
                new_EAs_history.append(EAs_history[idx])
                new_best_arch_history.append(best_arch_history[idx])
                new_best_performance_history.append(best_performance_history[idx])
            new_search_cost_history = search_cost_history[new_nEvals_history - 1]

            nEvals_list.append(new_nEvals_history)
            search_cost_list.append(new_search_cost_history)
            EAs_list.append(new_EAs_history)
            IGD_list.append(new_IGD_history)
            IGDp_list.append(new_IGDp_history)
            HV_list.append(new_HV_history)
            best_arch_list.append(new_best_arch_history)
            best_performance_list.append(new_best_performance_history)

        nEvals_list = np.array(nEvals_list)
        search_cost_list = np.array(search_cost_list)
        IGD_list = np.array(IGD_list)
        IGDp_list = np.array(IGDp_list)
        HV_list = np.array(HV_list)
        best_arch_list = np.array(best_arch_list)
        best_performance_list = np.array(best_performance_list)

        rs_ = {
            'nEvals': nEvals_list,
            'Search Cost': search_cost_list,
            'EAs': EAs_list,
            'IGD': IGD_list,
            'IGD+': IGDp_list,
            'HV': HV_list,
            'Best Architecture': best_arch_list,
            'Best Architecture (performance)': best_performance_list
        }
        p.dump(rs_, open(f'{PATH_RESULTS}/{problem}/{variant}_evaluation.p', 'wb'))

def summarize_res_search_phase(list_variants):
    for variant in list_variants:
        path_file = f'{PATH_RESULTS}/{problem}/{problem}_{variant}'

        nEvals_list = []
        search_cost_list = []

        EAs_list = []
        IGD_list = []
        IGDp_list = []
        HV_list = []
        best_arch_list = []
        best_performance_list = []

        for rid in range(31):
            EAs_history = []
            best_arch_history = []
            best_performance_history = []

            p_file_ = path_file + f'/{rid}'

            try:
                nEvals_history, EA_evaluate_history = p.load(open(p_file_ + '/#Evals_and_Elitist_Archive_search.p', 'rb'))
                _, IGD_history = p.load(open(p_file_ + '/#Evals_and_IGD_search.p', 'rb'))
                _, IGDp_history = p.load(open(p_file_ + '/#Evals_and_IGDp_search.p', 'rb'))
                _, HV_history = p.load(open(p_file_ + '/#Evals_and_HV_search.p', 'rb'))
            except FileNotFoundError:
                raise FileNotFoundError('Please download the prepared results!')
            search_cost_history = p.load(open(p_file_ + '/running_time.p', 'rb'))

            # Add the final evaluation_mark (i.e., 3000)
            if nEvals_history[-1] != len(search_cost_history):
                nEvals_history.append(len(search_cost_history))
                EA_evaluate_history.append(EA_evaluate_history[-1])
                IGD_history.append(IGD_history[-1])
                IGDp_history.append(IGDp_history[-1])
                HV_history.append(HV_history[-1])

            nEvals_history = np.array(nEvals_history)
            IGD_history = np.array(IGD_history)
            IGDp_history = np.array(IGDp_history)
            HV_history = np.array(HV_history)
            search_cost_history = np.array(search_cost_history)

            for EA in EA_evaluate_history:
                F = np.round(EA['F'], 6)
                ea = {
                    'Approximation Set': EA['X'],
                    'hashKey': EA['hashKey'],
                    'Approximation Front': F
                }
                idx_best_arch = np.argmin(F[:, 0])
                EAs_history.append(ea)
                best_arch = EA['X'][idx_best_arch]
                best_performance = np.round((1 - EA['F'][idx_best_arch][0]) * 100, 2)
                best_arch_history.append(best_arch)
                best_performance_history.append(best_performance)

            new_nEvals_history = np.arange(20, 3001, 20)
            new_IGD_history = []
            new_IGDp_history = []
            new_HV_history = []
            new_EAs_history = []
            new_best_arch_history = []
            new_best_performance_history = []
            for nEvals in new_nEvals_history:
                idx = np.where(nEvals_history <= nEvals)[0][-1]
                new_IGD_history.append(IGD_history[idx])
                new_IGDp_history.append(IGDp_history[idx])
                new_HV_history.append(HV_history[idx])
                new_EAs_history.append(EAs_history[idx])
                new_best_arch_history.append(best_arch_history[idx])
                new_best_performance_history.append(best_performance_history[idx])
            new_search_cost_history = search_cost_history[new_nEvals_history - 1]

            nEvals_list.append(new_nEvals_history)
            search_cost_list.append(new_search_cost_history)
            EAs_list.append(new_EAs_history)
            IGD_list.append(new_IGD_history)
            IGDp_list.append(new_IGDp_history)
            HV_list.append(new_HV_history)
            best_arch_list.append(new_best_arch_history)
            best_performance_list.append(new_best_performance_history)

        nEvals_list = np.array(nEvals_list)
        search_cost_list = np.array(search_cost_list)
        IGD_list = np.array(IGD_list)
        IGDp_list = np.array(IGDp_list)
        HV_list = np.array(HV_list)
        best_arch_list = np.array(best_arch_list)
        best_performance_list = np.array(best_performance_list)

        rs_ = {
            'nEvals': nEvals_list,
            'Search Cost': search_cost_list,
            'EAs': EAs_list,
            'IGD': IGD_list,
            'IGD+': IGDp_list,
            'HV': HV_list,
            'Best Architecture': best_arch_list,
            'Best Architecture (performance)': best_performance_list
        }
        p.dump(rs_, open(f'{PATH_RESULTS}/{problem}/{variant}_search.p', 'wb'))

## Visualization

In [ ]:
label = {
    'MOEA_NSGAII': 'NSGA-II',
    'LOMONAS': 'LOMONAS (ours)',
    'MOEA_MOEAD': 'MOEA/D',
    'RR_LS': 'RR-LS',
}

color = {
    'NSGA-II': ['blue', '--'],
    'MOEA/D': ['green', '-.'],
    'LOMONAS (ours)': ['tab:red', 'solid'],
    'RR-LS': ['tab:orange', '--'],
}

def visualize_2D(ax, objective_0_mean, tmp_objective_1_mean, tmp_objective_1_stdev, label, line):
    color = line[0]
    style = line[1]
    ax.plot(objective_0_mean, tmp_objective_1_mean, c=color, ls=style, label=label, linewidth=2)
    ax.fill_between(objective_0_mean,
                     tmp_objective_1_mean - tmp_objective_1_stdev,
                     tmp_objective_1_mean + tmp_objective_1_stdev, alpha=0.1, fc=color)

def visualize_results(ax, phase='evaluation', start_pt=20, end_pt=-1, metric='HV'):
    performance_algo_list = {}
    nEvals_algo_list = {}

    for variant in variants_list:
        results = p.load(open(f'{PATH_RESULTS}/{problem}/{variant}_{phase}.p', 'rb'))

        nEvals_all = results['nEvals'][0]
        idx_start = np.where(nEvals_all <= start_pt)[0][-1]

        if end_pt == -1:
            idx_end = None
        else:
            idx_end = np.where(nEvals_all <= end_pt)[0][-1]

        if metric == 'HV':
            perf = results['HV']
        elif metric == 'IGD':
            perf = results['IGD']
        else:
            perf = results['IGD+']

        if idx_end is not None:
            perf_mean, perf_std = np.mean(perf, axis=0)[idx_start:idx_end + 1], np.std(perf, axis=0)[idx_start:idx_end + 1]
            nEvals = nEvals_all[idx_start:idx_end + 1]
        else:
            perf_mean, perf_std = np.mean(perf, axis=0), np.std(perf, axis=0)
            nEvals = nEvals_all
        performance_algo_list[variant] = [perf_mean, perf_std]
        nEvals_algo_list[variant] = nEvals

    perf_ind = performance_algo_list

    ends = [100, 500, 1000, 3000]
    for variant in variants_list:
        visualize_2D(ax, nEvals_algo_list[variant], perf_ind[variant][0], perf_ind[variant][1], label=label[variant], line=color[label[variant]])
    ax.set_xscale('log')
    ax.set_xticks(ends)
    ax.get_xaxis().set_major_formatter(FormatStrFormatter('%2d'))
    ax.get_yaxis().set_major_formatter(FormatStrFormatter('%.3f'))

    ax.set_ylabel(f'{title}')
    ax.set_xlabel('#Evals')
    ax.grid(True, linestyle='--')
    ax.set_title(f'{metric} ({phase})', fontsize=24)
    if metric == 'HV':
        ax.legend(loc=4)
    else:
        ax.legend(loc=1)
    plt.savefig(f'{PATH_RESULTS}/{problem}/{metric}_{phase}.jpg', bbox_inches='tight', pad_inches=0.1, dpi=300)
    plt.show()

## T-Test

In [ ]:
def return_all_info(rs, comparison_point):
    nEvals_all = rs['nEvals'][0]
    if comparison_point == -1:
        idx = -1
    else:
        idx = np.where(nEvals_all <= comparison_point)[0][-1]

    IGD_lst = rs['IGD'][:, idx]
    IGDp_lst = rs['IGD+'][:, idx]
    HV_lst = rs['HV'][:, idx]
    running_time_lst = rs['Search Cost'][:, idx]
    best_arch_lst = rs['Best Architecture (performance)'][:, idx]
    return IGD_lst, IGDp_lst, HV_lst, running_time_lst, best_arch_lst

def compare(x, y, name_x, name_y, alpha, metric='IGD'):
    p_value = stats.ttest_ind(x, y)[-1]

    print(metric, name_x, name_y)
    print('p_value:', p_value)

    if np.isnan(p_value):
        p_value = 1

    if p_value <= alpha:
        mean_1 = np.mean(x)
        mean_2 = np.mean(y)
        std_1 = np.std(x)
        std_2 = np.std(y)
        cohen_d = (abs(mean_1 - mean_2)) / ((std_1 ** 2 + std_2 ** 2) / 2) ** (1 / 2)
        if cohen_d >= 0.8:
            effect_size = 'large'
        elif cohen_d >= 0.5:
            effect_size = 'medium'
        elif cohen_d >= 0.2:
            effect_size = 'small'
        else:
            effect_size = 'trivial'
        rs = np.mean(y) - np.mean(x)
        if metric in ['HV', 'best_arch_found']:
            rs *= -1
        if rs > 0:
            rs_compare = 'worse'
        else:
            rs_compare = 'better'
        print('Reject |', rs_compare, effect_size)
    print()

def print_mean_std(_all, round_pre=4, print_std=True):
    _mean = np.mean(_all, axis=0)
    _std = np.std(_all, axis=0)
    if print_std:
        print(f'{np.round(_mean, round_pre)} ({np.round(_std, round_pre)})')
    else:
        print(f'{np.round(_mean, round_pre)}')

def print_interval_confidence(_all, round_pre=4):
    _mean = np.mean(_all, axis=0)
    _std = np.std(_all, axis=0)
    tmp = 3.657 * _std/(len(_all) ** (1/2))
    print(f'({np.round(_mean - tmp, round_pre)} {np.round(_mean + tmp, round_pre)}) | {np.round(_mean, round_pre)}')


def cal_interval_confidence(variants_lst, phase):
    alpha = 0.01
    print(f'Alpha: {alpha/len(variants_lst)}')

    for variant in variants_lst:
        print(variant)
        rs = p.load(open(f'{PATH_RESULTS}/{problem}/{variant}_{phase}.p', 'rb'))
        igd, igdp, hv, rt, best_arch = return_all_info(rs, -1)

        print_interval_confidence(igd)
        print_interval_confidence(igdp)
        print_interval_confidence(hv)

def t_test(variants_lst, phase, comparison_point):
    alpha = 0.01/len(variants_lst)
    print(f'Alpha: {alpha}')
    print(f'Evals: {comparison_point}\n')
    competitors_lst = []
    for variant in variants_lst:
        variant_ = variant + f'_{phase}'
        if pathlib.Path(f'{PATH_RESULTS}/{problem}/{variant_}.p').exists():
            print(variant)
            rs = p.load(open(f'{PATH_RESULTS}/{problem}/{variant_}.p', 'rb'))
            igd, igdp, hv, rt, best_arch = return_all_info(rs, comparison_point)
            print('IGD: ', end='')
            print_mean_std(igd)
            print('IGD+: ', end='')
            print_mean_std(igdp)
            print('HV: ', end='')
            print_mean_std(hv)
            print('Running time (in seconds): ', end='')
            print_mean_std(rt, 0, False)
            print('Best Architecture: ', end='')
            print_mean_std(best_arch, 2)

            performance = {
                'variant': variant,
                'IGD': igd,
                'IGD+': igdp,
                'HV': hv,
                'best_arch_found': best_arch
            }
            competitors_lst.append(performance)
            print('-' * 10)
    print('='*20)
    print()
    for metric in ['IGD', 'IGD+', 'HV']:
        print('Metric:', metric)
        for i in range(len(competitors_lst)):
            for j in range(i + 1, len(competitors_lst)):
                pair = [competitors_lst[i][metric], competitors_lst[j][metric],
                        competitors_lst[i]['variant'], competitors_lst[j]['variant'], alpha, metric]
                compare(*pair)
        print('='*20)
        print()

## Main

### Summarize results and Visualize

In [ ]:
PATH_RESULTS = '/content/LOMONAS/exp_res'  # Your experiment result path
problem = 'NAS201-C10'

print(problem)
if problem == 'NAS201-C10':
    title = 'NAS-201 (CIFAR-10)'
elif problem == 'NAS201-C100':
    title = 'NAS-201 (CIFAR-100)'
elif problem == 'NAS201-IN16':
    title = 'NAS-201 (IN16-120)'
elif problem == 'NAS101':
    title = 'NAS-Bench-101'
elif problem == 'MacroNAS-C10':
    title = 'MacroNAS (CIFAR-10)'
elif problem == 'NAS-ASR':
    title = 'NAS-Bench-ASR'
else:
    title = 'MacroNAS (CIFAR-100)'

################################################ Summarize Results #################################################
variants_list = [
    'MOEA_NSGAII',
    'MOEA_MOEAD',
    'RR_LS',
    'LOMONAS'
]

# Summarize results
summarize_res_search_phase(variants_list)
summarize_res_evaluation_phase(variants_list)

################################################ Visualize Results #################################################
for phase in ['search', 'evaluation']:
    for i, metric in enumerate(['IGD', 'IGD+', 'HV']):
        fig, ax = plt.subplots(figsize=(10, 6))
        visualize_results(ax=ax, phase=phase, start_pt=100, end_pt=3000, metric=metric)

### Run statitical tests

In [ ]:
for phase in ['search', 'evaluation']:
    print('Phase:', phase.upper())
    variants_list = [
        'MOEA_NSGAII',
        'MOEA_MOEAD',
        'RR_LS',
        'LOMONAS'
    ]

    t_test(variants_list, phase=phase, comparison_point=3000)